In [1]:
#
# The MIT License (MIT)

# Copyright (c) 2021, NVIDIA CORPORATION

# Permission is hereby granted, free of charge, to any person obtaining a copy of
# this software and associated documentation files (the "Software"), to deal in
# the Software without restriction, including without limitation the rights to
# use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
# the Software, and to permit persons to whom the Software is furnished to do so,
# subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
# FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
# COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
# IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
# CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
#

In [2]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import cudf
import cupy
import nvtabular as nvt
import json

In [2]:
nvt.__version__

'0.5.1+37.g6c2de9b'

In [3]:
DATA_FOLDER = "/workspace/"
FILENAME_PATTERN_BROWSING = 'browsing_train.csv'
FILENAME_PATTERN_SEARCH = 'search_train.csv'
DATA_PATH_BROWSING = os.path.join(DATA_FOLDER, FILENAME_PATTERN_BROWSING)
DATA_PATH_SEARCH = os.path.join(DATA_FOLDER, FILENAME_PATTERN_SEARCH)
OUTPUT_DIR = "/workspace/coveo_task1_v4"
!ls $DATA_PATH_BROWSING

/workspace/browsing_train.csv


In [4]:
MINIMUM_SESSION_LENGTH = 2

**5 steps: each section is independant and create different parquet files:**

   - Pandas processing: <a href ='#pandas_proc'> Section 1 </a> 
       - <a href='#search_table'>Section 1.1</a>:  Process search table and create session_search table that groups query vectors and impressions seen/clicked within the same sessions. 
       - <a href='#event_table'>Section 1.2 </a>: Create event table by merging browsing, test and search data 
       
   
   - Create preprocessed interactions table:  <a href='#row_workflow'> Section 2 </a>
           - Filling missing values 
           - Convert timestamps 
           - Categorify and Normalize variables
   
   - Create preprocess session table:  <a href='#session_workflow'> Section 3 </a>
   
   - Merge session_browsing and session_search tables 

# <a id='pandas_proc'>Processing DATA with pandas </a>

## <a id='event_table'> Preprocessing of search tables: merging Train and Test data </a>

In [5]:
# load search data
search = pd.read_csv(DATA_PATH_SEARCH, sep=',')
# Add column event_type 
search['event_type'] = 'search'
# Add column 'is_search'
search['is_search'] = 1
search['is_test'] = 0
# drop 123 rows where: (clicked_skus_hash != NaN) and (product_skus_hash == NaN)
condition = (search['product_skus_hash'].isnull()) & (~search['clicked_skus_hash'].isnull())
search = search.loc[~condition]
# convert strings to list object 
import ast
def convert_str_to_list(x): 
    if pd.isnull(x): 
        return x
    return ast.literal_eval(x)
for col in ['product_skus_hash', 'clicked_skus_hash', 'query_vector']: 
    search[col] = search[col].progress_apply(convert_str_to_list)

# Add search events from test data
# load test data 
with open('/workspace/rec_test_phase_1.json') as json_file:
    # read the test cases from the provided file
    test_queries = json.load(json_file)
test_df = pd.json_normalize(test_queries, 'query')
test_df['is_test'] = 1
test_search = test_df[['session_id_hash', 'query_vector', 'clicked_skus_hash',
       'product_skus_hash', 'server_timestamp_epoch_ms', 'event_type',
       'is_search', 'is_test']]
test_search = test_search[test_search.is_search==True]

# concat test and train search data
search = pd.concat([search, test_search])
search.reset_index(inplace=True)

100%|██████████| 819393/819393 [02:56<00:00, 4635.12it/s]


In [6]:
del test_search

- **Compute the number of search queries per session** 

In [7]:
tmp = search.groupby('session_id_hash').size().reset_index()
tmp.columns = ['session_id_hash', 'nb_queries']
search = search.merge(tmp, on='session_id_hash', how='left')

* **Include unseen clicked product to impression list** : 

    2% of the search events have a clicked item that does not appear in the impression list. 
        ==> We add the missing elements to the impression list. 

In [8]:
def add_clicked(x): 
    if isinstance(x.clicked_skus_hash, list) and isinstance(x.product_skus_hash, list):
        return list(set(x.product_skus_hash).union(set(x.clicked_skus_hash)))
    return x.product_skus_hash
search['updated_product_skus_hash'] = search.progress_apply(add_clicked, axis=1)

100%|██████████| 836312/836312 [00:30<00:00, 27321.43it/s]


* **Compute the number of visualized products and clicked items per search** 

In [9]:
search['impression_size'] = search.product_skus_hash.str.len().fillna(0)
search['clicks_size'] = search.clicked_skus_hash.str.len().fillna(0)

In [10]:
search.head(2)

,index,session_id_hash,query_vector,clicked_skus_hash,product_skus_hash,server_timestamp_epoch_ms,event_type,is_search,is_test,nb_queries,updated_product_skus_hash,impression_size,clicks_size
0,0,48fade624d47870058ce07dd789ccc04e46c70c0fa2a1b...,"[-0.20255649089813232, -0.016908567398786545, ...",NaN,NaN,1548575194779,search,1,0,1,NaN,0.0,0.0
1,1,8731ca84ff7bb8cb647531d54e64feedb2519b4a7792a7...,"[-0.007610442116856575, -0.14909175038337708, ...",NaN,[9ee9ffd7e2529a65f9a0b0c9eaae6330df85cf2e3af33...,1548276763869,search,1,0,1,[9ee9ffd7e2529a65f9a0b0c9eaae6330df85cf2e3af33...,3.0,0.0


- **Aggregate search information per session** 

The resulting variables are: 

    - "flat_query_vector": List of search query vector (reshaped to 1-dim so that it works with NVT Data loader)
    - "flat_product_skus_hash": List of product skus returned by the search query
    - "clicked-flag": To indicate if the product was clicked or not in the search results
    - "nb_queries" : number of search queries within the same session 
    - "impression_size": number of returned product skus for each search query
    - "clicks_size": number of clicked items for each search query



In [11]:
def all_products(x): 
    t =[]
    for products in x.dropna(): 
        t += products
    if len(t)==0:
        return ['missing']
    return t

session_search = search.sort_values(['session_id_hash', 'server_timestamp_epoch_ms']).groupby('session_id_hash').agg({'query_vector': lambda x: list(np.concatenate(x.values)),
                                                                                                                    'updated_product_skus_hash': all_products,
                                                                                                                    'clicked_skus_hash': all_products,
                                                                                                                    'impression_size': list,
                                                                                                                    'clicks_size': list,
                                                                                                                    'nb_queries': 'last'
                                                                                                                  })
session_search.columns = ['flat_query_vector', 'flat_product_skus_hash', 'flat_clicked_skus_hash', 'impressions_size', 'clicks_size', 'nb_queries']
session_search['clicked-flag'] = session_search.progress_apply(lambda x: [int(e in x['flat_clicked_skus_hash']) for e in x['flat_product_skus_hash']], axis=1)
session_search = session_search.reset_index()

100%|██████████| 562208/562208 [00:58<00:00, 9538.81it/s] 


In [12]:
session_search.head(3)

,session_id_hash,flat_query_vector,flat_product_skus_hash,flat_clicked_skus_hash,impressions_size,clicks_size,nb_queries,clicked-flag
0,00000277639fc5c6f816654b78bf3654ece7fd53a7338f...,"[-0.1088152825832367, -0.09937708079814911, -0...",[d9550a87fde6e944796797375ab71e01a08323f9916ac...,[ca089db21669aabe32904308205ad3632574fd268e912...,[14.0],[1.0],1,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,00001216514808fdd1e3bd4e4229ef518c712932424d82...,"[-0.05604613572359085, -0.19420677423477173, 0...",[e90aeee7e8eb921bd430981037ee19c6e95226d0651c6...,[missing],[25.0],[0.0],1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,000049b258d564d8a44105a5c85b1a82e69bae9129319b...,"[-0.018799802288413048, -0.2817137837409973, 0...",[7302590bebd27fb6dece6dc605b96a098ab1efdf35aff...,[missing],"[0.0, 21.0]","[0.0, 0.0]",2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


- **Save search tables** 

In [13]:
session_search.to_parquet(os.path.join(OUTPUT_DIR, "session_search.parquet"))
search.to_parquet(os.path.join(OUTPUT_DIR, "search.parquet"))

## <a id='event_table'> Create event table: merging browsing, test and search data </a>

In [15]:
session_search = pd.read_parquet(os.path.join(OUTPUT_DIR, "session_search.parquet"))
search = pd.read_parquet(os.path.join(OUTPUT_DIR, "search.parquet"))

In [16]:
browsing = pd.read_csv(DATA_PATH_BROWSING, sep=',')
# Add columns 'is_search' ans 'is_test'
browsing['is_search'] = 0 
browsing['is_test'] = 0 

- **Create event table**: mergin browsing_train and test data

In [17]:
with open('/workspace/rec_test_phase_1.json') as json_file:
    # read the test cases from the provided file
    test_queries = json.load(json_file)
test_df = pd.json_normalize(test_queries, 'query')
test_df['is_test'] = 1

In [18]:
# concat browsing and search data to create event table 
event_df = pd.concat([browsing, test_df])
event_df.reset_index(drop=True, inplace=True)

In [19]:
event_df.head(3)

,session_id_hash,event_type,product_action,product_sku_hash,server_timestamp_epoch_ms,hashed_url,is_search,is_test,query_vector,clicked_skus_hash,product_skus_hash
0,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,d5157f8bc52965390fa21ad5842a8502bc3eb8b0930f3f...,1550885210881,7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d...,0,0,NaN,NaN,NaN
1,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,61ef3869355b78e11011f39fc7ac8f8dfb209b3442a9d5...,1550885213307,4ed279f4f0deab6dfc80f4f7bf49d527fd894fa478a9ce...,0,0,NaN,NaN,NaN
2,20c458b802f6ea9374783bfc528b19421be977a6769785...,pageview,NaN,NaN,1550885213307,4ed279f4f0deab6dfc80f4f7bf49d527fd894fa478a9ce...,0,0,NaN,NaN,NaN


In [20]:
event_df.tail(3)

,session_id_hash,event_type,product_action,product_sku_hash,server_timestamp_epoch_ms,hashed_url,is_search,is_test,query_vector,clicked_skus_hash,product_skus_hash
36655739,fffe75a4f9e8b6dead0825d7db7fd538a64bc117599030...,event_product,detail,4e880bf467dcb699fd61519bc3845dd6524b2095d45e7f...,1557406442986,c3eb53ea31db5ba124449624561969376aceb63297ce18...,0,1,None,None,None
36655740,ffff39e95bd7ebc272897b54865d60accb111e0ec42cc0...,event_product,detail,3a62198d671b861ee178dd025f321ac66c28ba2e6967c6...,1557337655966,d4bb262e543866324b517cf4e294c77d270cfabe5d5f4f...,0,1,None,None,None
36655741,ffffccb130241d462149241caf4f17a45d038481e51c28...,event_product,detail,caf9a5bfd81ef43f6cf3cbdc6017fdc79d43bcdea9fdb2...,1556234499481,4a2063eada9db11971c1264b28296cae1eaa133a395578...,0,1,None,None,None


In [21]:
del browsing

- **Process duplicated events:** which are defined as interactions that occur in the same session and at the same time 

In [22]:
# drop duplicated rows (total of 1800) 
event_df.drop_duplicates(['session_id_hash' ,'server_timestamp_epoch_ms', 'event_type', 'product_action', 'product_sku_hash', 'hashed_url'],inplace=True, keep='last')
# Remove indices of 'pageview' interactions from duplicated events defined above 
tmp = event_df[(event_df.event_type == 'pageview') & (event_df.duplicated(['session_id_hash' , 'server_timestamp_epoch_ms']))]
event_df.drop(tmp.index, inplace=True)

In [23]:
event_df.shape

(31769147, 11)

* **Add search events**: 

In [24]:
# select search events with clicks
use_cols = ['session_id_hash', 'clicked_skus_hash',
            'server_timestamp_epoch_ms', 'event_type',
            'is_search', 'is_test']
search_clicks = search[search.clicks_size>0][use_cols]

# specify event type and product action
search_clicks['event_type'] = 'search'
search_clicks['product_action'] = 'click'

#unstack list of clicked items to multiple rows
lst_col = 'clicked_skus_hash'
search_clicks = pd.DataFrame({
    col:np.repeat(search_clicks[col].values, search_clicks[lst_col].str.len()) for col in search_clicks.columns.difference([lst_col])}).assign(
    **{lst_col:np.concatenate(search_clicks[lst_col].values)})[search_clicks.columns.tolist()]

search_clicks.columns = ['session_id_hash', 'product_sku_hash', 'server_timestamp_epoch_ms',
                         'event_type', 'is_search', 'is_test', 'product_action']

In [25]:
search_clicks.head(3)

,session_id_hash,product_sku_hash,server_timestamp_epoch_ms,event_type,is_search,is_test,product_action
0,77d604ce112a107ffe3e60566778d090a0b756bcfb0a3e...,20e8f1c4051a6c55122bd00f25f92f91b5f125497b7939...,1549078957672,search,1,0,click
1,7fc129b5945b2959339da1c0675d9c9adcdbc1fc99a954...,8283e8b59c283c3b72462a6f73084683524f4b0149acb7...,1550003058242,search,1,0,click
2,3ef3f3170f85ae92b01378b2161f5e71f64caadbeefd90...,2901d00c55fc2e4e60ea0d0e7a48ee843491e31025afab...,1549641526668,search,1,0,click


In [26]:
event_df = pd.concat([event_df, search_clicks])
event_df.event_type.value_counts()

pageview         21077946
event_product    10674282
search             398727
Name: event_type, dtype: int64

* **Add product information**

In [27]:
product_info = pd.read_csv('/workspace/sku_to_content.csv')
product_info.tail()

,product_sku_hash,description_vector,category_hash,image_vector,price_bucket
66381,c7cc673ca3baa5fa222fffdc16379892b3a62583a48143...,NaN,NaN,NaN,NaN
66382,6641c7d2053ce48ce1e81a9653dffe56dbb79ab0704fbd...,"[-0.19150441884994507, -0.06235162168741226, -...",0665a81d19c89281cc00e7f7d779ded2ed42c933838602...,"[129.55668732976045, 43.27996741934932, -36.70...",3.0
66383,526a6a51717d5bb40ef2b0c47394d08c54385375633bab...,"[-0.19760936498641968, 0.4446450471878052, -0....",0665a81d19c89281cc00e7f7d779ded2ed42c933838602...,"[-158.25984189321855, 74.19255741438077, 199.6...",5.0
66384,21ca4ab0e2fbd3b401fbeadeb4439dcab9998fb52159ec...,NaN,NaN,NaN,NaN
66385,0eaa39fb645749da181c0637d3b420f3f103b5f6b50286...,NaN,NaN,NaN,NaN


- Extract product main category 

In [28]:
def product_main_category(x):
    if pd.isna(x):
        return x
    return x.split('/')[0]
product_info['main_category'] = product_info['category_hash'].progress_apply(product_main_category)

100%|██████████| 66386/66386 [00:00<00:00, 321180.65it/s]


- Compute average price of main and hierarchy category

In [29]:
main_price = product_info.groupby('main_category')['price_bucket'].mean().reset_index()
main_price.columns = ['main_category', 'mean_price_main']

hierarchy_price = product_info.groupby('category_hash')['price_bucket'].mean().reset_index()
hierarchy_price.columns = ['category_hash', 'mean_price_hierarchy']

- Merge with product table 

In [30]:
product_info = product_info.merge(main_price, on=['main_category'], how='left')
product_info = product_info.merge(hierarchy_price, on=['category_hash'], how='left')

In [31]:
product_info.head(3)

,product_sku_hash,description_vector,category_hash,image_vector,price_bucket,main_category,mean_price_main,mean_price_hierarchy
0,26ce7b47f4c46e4087e83e54d2f7ddc7ea57862fed2e2a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6383992be772b204a9ab75f86c86f5583d1bdd1222952d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a2c3e2430c6ef9770b903ad08fa067a6b2b9db28f06e1b...,"[0.27629122138023376, -0.15763211250305176, 0....",06fa312761d4b39e2f649781514ac69a4c1505c221fc46...,"[340.3592564184389, -220.19025864725685, 154.0...",7.0,06fa312761d4b39e2f649781514ac69a4c1505c221fc46...,5.69263,7.859416


* Merge product table and event_df 

In [32]:
event_df = event_df.merge(product_info[['product_sku_hash', 'main_category', 'category_hash',
                                        'price_bucket', 'mean_price_hierarchy', 'mean_price_main' ]], on='product_sku_hash', how='left')

In [33]:
event_df.head(2)

,session_id_hash,event_type,product_action,product_sku_hash,server_timestamp_epoch_ms,hashed_url,is_search,is_test,query_vector,clicked_skus_hash,product_skus_hash,main_category,category_hash,price_bucket,mean_price_hierarchy,mean_price_main
0,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,d5157f8bc52965390fa21ad5842a8502bc3eb8b0930f3f...,1550885210881,7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d...,0,0,NaN,NaN,NaN,0665a81d19c89281cc00e7f7d779ded2ed42c933838602...,0665a81d19c89281cc00e7f7d779ded2ed42c933838602...,10.0,9.857143,5.009099
1,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,61ef3869355b78e11011f39fc7ac8f8dfb209b3442a9d5...,1550885213307,4ed279f4f0deab6dfc80f4f7bf49d527fd894fa478a9ce...,0,0,NaN,NaN,NaN,0665a81d19c89281cc00e7f7d779ded2ed42c933838602...,0665a81d19c89281cc00e7f7d779ded2ed42c933838602...,10.0,9.857143,5.009099


* **Remove duplicated interactions with the same product**: encode original information as binary flags

In [34]:
product_interactions = event_df.groupby(['session_id_hash', 'product_sku_hash']).agg({
    'product_sku_hash': len,
    'product_action': [lambda x: 'add' in list(x), 
                       lambda x: 'detail' in list(x),
                       lambda x: 'purchase' in list(x),
                       lambda x: 'remove' in list(x),
                       lambda x: 'click' in list(x),
                      ]
}).droplevel(0, axis=1)

In [35]:
product_interactions = product_interactions.reset_index()

In [36]:
product_interactions.columns = ['session_id_hash', 'product_sku_hash', 'nb_interactions',
                                'has_been_added_to_cart', 'has_been_detailed', 
                                'has_been_purchased', 'has_been_removed_from_cart', 'has_been_clicked' ]

In [37]:
product_interactions.head(2)

,session_id_hash,product_sku_hash,nb_interactions,has_been_added_to_cart,has_been_detailed,has_been_purchased,has_been_removed_from_cart,has_been_clicked
0,00000114e1075962f022114fcfc17f2d874e694ac5d201...,4945f2fa8e87cb7501702ed3dce26253296eae7a8f670f...,1,False,True,False,False,False
1,00000114e1075962f022114fcfc17f2d874e694ac5d201...,6ff8d0f30bbe66cfec7d87fc7e22bd8b1defd47ff4aaaa...,1,False,True,False,False,False


-  **Remove repeated interactions of event-product to align with private test set** 

In [38]:
event_df = event_df.sort_values(['session_id_hash', 'server_timestamp_epoch_ms'])

In [39]:
event_df = event_df.drop_duplicates(['session_id_hash','event_type','product_sku_hash','hashed_url'], keep='first')

In [40]:
event_df.shape

(22364905, 16)

*  **Merge product interactions info and event df** 

In [41]:
event_df = event_df.merge(product_interactions, on=['session_id_hash', 'product_sku_hash'], how='left')

In [42]:
event_df.shape

(22364905, 22)

In [43]:
event_df.head(2)

,session_id_hash,event_type,product_action,product_sku_hash,server_timestamp_epoch_ms,hashed_url,is_search,is_test,query_vector,clicked_skus_hash,...,category_hash,price_bucket,mean_price_hierarchy,mean_price_main,nb_interactions,has_been_added_to_cart,has_been_detailed,has_been_purchased,has_been_removed_from_cart,has_been_clicked
0,00000114e1075962f022114fcfc17f2d874e694ac5d201...,pageview,NaN,NaN,1552423391039,0aa1084eddfb08e4dffbb5a2aa98a5e9679382d982dd97...,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00000114e1075962f022114fcfc17f2d874e694ac5d201...,event_product,detail,cf2f88cb43c1713538f7dfd2aa498a2cb9ebc0c99feeac...,1552423391039,0aa1084eddfb08e4dffbb5a2aa98a5e9679382d982dd97...,0,0,NaN,NaN,...,06fa312761d4b39e2f649781514ac69a4c1505c221fc46...,10.0,9.469945,5.69263,3.0,True,True,False,False,False


In [44]:
del product_interactions

*  **Add column product_url_hash**: SKUs are filled with hashed_url for pageview events

In [45]:
event_df['product_url_hash'] = event_df['product_sku_hash'].fillna(event_df['hashed_url'])

### Save into parquet files 

In [46]:
event_df.event_type.value_counts()

pageview         14210232
event_product     7883166
search             259327
Name: event_type, dtype: int64

In [47]:
# save to parquet file with 40 partitions 
event_df['parquet_split'] =  np.random.randint(0,40,size=event_df.shape[0])
event_df.to_parquet(os.path.join(OUTPUT_DIR, "event_train_v4"), partition_cols=['parquet_split'])

In [48]:
import gc
del event_df
gc.collect()

17006

# <a id='row_workflow'> Define the preprocessed row interactions table </a>

In [49]:
import glob
files = glob.glob(OUTPUT_DIR + '/event_train_v4/parquet_split*/*.parquet')

- **1st workflow: fill missing values, encode categorical variables and normalize numericals**

In [52]:
#  load data 
df_event = nvt.Dataset(files, part_size="1GB") 

# convert timestamp to datetime object
to_datetime = ["server_timestamp_epoch_ms"] >> nvt.ops.LambdaOp(lambda col: cudf.to_datetime(col, unit='ms')) >> nvt.ops.Rename( f = lambda x: 'timestamp')

#categorical features
# fill missing product_actions
missing_action = ['product_action'] >> nvt.ops.FillMissing(fill_val='view')  >> nvt.ops.Rename(postfix = '_filled')
categ_feats = ['session_id_hash',  'event_type', 'price_bucket', 'main_category', 'category_hash']
cat_feats = missing_action + categ_feats >> nvt.ops.Categorify()

# fill missing product ids before categorify to keep id '0' for padding 
missing_ids = ['product_sku_hash','hashed_url', 'product_url_hash' ] >> nvt.ops.FillMissing(fill_val='missing')
cat_product_ids = missing_ids >> nvt.ops.Categorify()

# numerical features 
cont_feats = ['mean_price_hierarchy', 'mean_price_main', 'nb_interactions'] >> nvt.ops.FillMedian()
continuous_feats = cont_feats >> nvt.ops.Normalize()

#bool features 
bool_feats = ['has_been_added_to_cart', 'has_been_detailed', 'has_been_purchased', 
              'has_been_removed_from_cart', 'has_been_clicked', 'is_search', 'is_test']
boolean_feats = bool_feats >> nvt.ops.FillMissing(fill_val=0)

In [53]:
workflow = nvt.Workflow(to_datetime + cat_feats + cat_product_ids + continuous_feats + boolean_feats)
workflow.fit(df_event)
new_gdf = workflow.transform(df_event).to_ddf().compute()
len(new_gdf)

22364905

In [54]:
new_gdf.columns

Index(['timestamp', 'product_action_filled', 'session_id_hash', 'event_type',
       'price_bucket', 'main_category', 'category_hash', 'product_sku_hash',
       'hashed_url', 'product_url_hash', 'mean_price_hierarchy',
       'mean_price_main', 'nb_interactions', 'has_been_added_to_cart',
       'has_been_detailed', 'has_been_purchased', 'has_been_removed_from_cart',
       'has_been_clicked', 'is_search', 'is_test'],
      dtype='object')

In [55]:
# save the workflow : 
workflow.save(os.path.join(OUTPUT_DIR, "categorify_workflow"))

- **Item recency:** include the item first time seen feature using product_url_hash column 

In [56]:
items_first_ts_df = new_gdf.groupby('product_url_hash').agg({'timestamp': 'min'}).reset_index().rename(columns={'timestamp': 'itemid_ts_first'})
interactions_merged_df = new_gdf.merge(items_first_ts_df, on=['product_url_hash'], how='left')

In [57]:
interactions_merged_df.shape

(22364905, 21)

In [58]:
interactions_merged_df.shape

(22364905, 21)

- **Save encoded interaction table to parquet file** 

In [59]:
use_cols = ['session_id_hash', 'timestamp',  'event_type', 'product_action_filled',
            
            'product_sku_hash','hashed_url', 'product_url_hash',
            
            'main_category', 'category_hash', 'price_bucket', 'mean_price_hierarchy', 'mean_price_main', 'itemid_ts_first', 
            
            'nb_interactions', 'has_been_added_to_cart', 'has_been_detailed', 'has_been_purchased', 'has_been_removed_from_cart', 'has_been_clicked',
            
            'is_search', 'is_test']

interactions_merged_df[use_cols].head(5)

,session_id_hash,timestamp,event_type,product_action_filled,product_sku_hash,hashed_url,product_url_hash,main_category,category_hash,price_bucket,...,mean_price_main,itemid_ts_first,nb_interactions,has_been_added_to_cart,has_been_detailed,has_been_purchased,has_been_removed_from_cart,has_been_clicked,is_search,is_test
0,227553,2019-01-16 20:18:36.251,2,6,58914,131410,131212,0,0,0,...,0.042575,2019-01-15 05:03:02.542,-0.170007,False,False,False,False,False,0,0
1,227568,2019-04-10 13:22:00.800,1,3,27896,325454,235281,1,7,3,...,-2.446339,2019-01-16 23:02:11.599,-0.170007,False,True,False,False,False,0,0
2,227573,2019-04-04 21:58:46.525,1,3,33671,456797,283556,2,63,1,...,0.042575,2019-01-15 20:46:17.448,-0.170007,False,True,False,False,False,0,0
3,227574,2019-03-09 01:21:26.305,2,6,58914,137281,137056,0,0,0,...,0.042575,2019-01-15 20:47:00.535,-0.170007,False,False,False,False,False,0,0
4,227598,2019-04-13 01:32:21.534,2,6,58914,86991,86888,0,0,0,...,0.042575,2019-01-15 19:44:39.637,-0.170007,False,False,False,False,False,0,0


In [60]:
interactions_merged_df[use_cols].to_parquet(os.path.join(OUTPUT_DIR, 'row_interactions_preproc_v4.parquet'))

# <a id='session_workflow'>Preprocessing of session table - V4 </a>: 

In [5]:
interactions_merged_df = cudf.read_parquet(os.path.join(OUTPUT_DIR, 'row_interactions_preproc_v4.parquet'))

In [6]:
OUTPUT_FOLDER = "/workspace/coveo_dataset/"
!mkdir -p $OUTPUT_FOLDER

In [7]:
#cont features
cont_feats = ['mean_price_hierarchy', 'mean_price_main', 'nb_interactions'] >> nvt.ops.FillMedian()
#bool features 
bool_feats = ['has_been_added_to_cart', 'has_been_detailed', 'has_been_purchased', 
              'has_been_removed_from_cart', 'has_been_clicked', 'is_search', 'is_test']
interactions_merged_df[bool_feats] = interactions_merged_df[bool_feats].astype('int32')

- **Define temporal features**

In [8]:
# create time features
sessionTime = ['timestamp']

sessionTime_hour = (
    sessionTime >> 
    #nvt.ops.LambdaOp(lambda col: cudf.to_datetime(col, unit='ms').dt.hour) >> 
    nvt.ops.LambdaOp(lambda col: col.dt.hour) >> 
    nvt.ops.Rename(postfix = '_hour')
)
sessionTime_weekday = (
    sessionTime >> 
    #nvt.ops.LambdaOp(lambda col: cudf.to_datetime(col, unit='ms').dt.weekday) >> 
    nvt.ops.LambdaOp(lambda col: col.dt.weekday) >> 
    nvt.ops.Rename(postfix = '_wd')
)
sessionTime_day = (
    sessionTime >> 
    nvt.ops.LambdaOp(lambda col: col.dt.day) >> 
    nvt.ops.Rename(postfix="_day")
)

sessionTime_timestamp = (
    sessionTime >> 
    nvt.ops.LambdaOp(lambda col: (col.astype(int) / 1e6).astype(int)) >> 
    nvt.ops.Rename(f = lambda col: "ts")
)

In [9]:
def get_cycled_feature_value_sin(col, max_value):
    value_scaled = (col + 0.000001) / max_value
    value_sin = np.sin(2*np.pi*value_scaled)
    return value_sin

def get_cycled_feature_value_cos(col, max_value):
    value_scaled = (col + 0.000001) / max_value
    value_cos = np.cos(2*np.pi*value_scaled)
    return value_cos

In [10]:
hour_sin = sessionTime_hour >> (lambda col: get_cycled_feature_value_sin(col, 24)) >> nvt.ops.Rename(postfix = '_sin')
hour_cos = sessionTime_hour >> (lambda col: get_cycled_feature_value_cos(col, 24)) >> nvt.ops.Rename(postfix = '_cos')
weekday_sin = sessionTime_weekday >> (lambda col: get_cycled_feature_value_sin(col+1, 7)) >> nvt.ops.Rename(postfix = '_sin')
weekday_cos= sessionTime_weekday >> (lambda col: get_cycled_feature_value_cos(col+1, 7)) >> nvt.ops.Rename(postfix = '_cos')

In [11]:
cycled_features = hour_sin + hour_cos + weekday_sin + weekday_cos

In [12]:
cycled_features.columns

['timestamp_hour_sin',
 'timestamp_hour_cos',
 'timestamp_wd_sin',
 'timestamp_wd_cos']

In [13]:
# calculate item recency 
# create custom op
from nvtabular.ops import Operator

class ItemRecency(Operator):
    def transform(self, columns, gdf):
        for column in columns:
            col = gdf[column]
            #col.loc[col == ""] = None
            item_first_timestamp = gdf['itemid_ts_first']
            delta_days = (col - item_first_timestamp).dt.days
            gdf[column + "_age_days"] = delta_days * (delta_days >=0)
        return gdf
            
    def output_column_names(self, columns):
        return [column + "_age_days" for column in columns]
            
    def dependencies(self):
        return ["itemid_ts_first"]

In [14]:
recency_features = ["timestamp"] >> ItemRecency() 
recency_features_norm = recency_features >> nvt.ops.LogOp() >> nvt.ops.Normalize() >> nvt.ops.Rename(postfix = '_norm')

In [15]:
recency_features.columns, recency_features_norm.columns
time_features = (
    sessionTime_timestamp +
    sessionTime + 
    sessionTime_hour +
    sessionTime_day + 
    sessionTime_weekday +
    recency_features +
    recency_features_norm + 
    cycled_features)

In [16]:
time_features.columns

['ts',
 'timestamp',
 'timestamp_hour',
 'timestamp_day',
 'timestamp_wd',
 'timestamp_age_days',
 'timestamp_age_days_norm',
 'timestamp_hour_sin',
 'timestamp_hour_cos',
 'timestamp_wd_sin',
 'timestamp_wd_cos']

- **Grouping interactions into sessions**

In [17]:
# Define Groupby Workflow: search columns are not used
# N.B: Add the op ListSlice when upgrading nvt 0.5.1 to 0.6 
filter_nan_products = (interactions_merged_df.columns >> nvt.ops.Filter(f=lambda df: df['product_sku_hash'] != 0))


groupby_only_product = filter_nan_products - ['timestamp']  + time_features  >> nvt.ops.Groupby(
    groupby_cols=["session_id_hash"], 
    sort_cols=["ts"],
    aggs={
       "product_sku_hash": ["list", "count"], 
    }
)
    
groupby_product_url = ['session_id_hash', 'product_url_hash']  + time_features >> nvt.ops.Groupby(
    groupby_cols=["session_id_hash"], 
    sort_cols=["ts"],
    aggs={
       "product_url_hash": ["list", "count"]
    }
)
groupby_other_features =  bool_feats + cont_feats + ['product_sku_hash', 'product_action_filled' ,'session_id_hash', 'event_type',
                                                                             'price_bucket', 'main_category', 'category_hash'] + time_features >> \
    nvt.ops.Groupby(
    groupby_cols=["session_id_hash"], 
    sort_cols=["ts"],
    aggs={
        "product_action_filled": ["list"],     
        "event_type": ["list"],    
        "price_bucket": ["list"],
        "category_hash": ["list"],
        'has_been_added_to_cart': ["list"],
        'has_been_detailed': ["list"],
        'has_been_purchased': ["list"],
        'has_been_removed_from_cart': ["list"],
        'has_been_clicked': ["list"],
        'mean_price_hierarchy':["list"],
        'mean_price_main':["list"],
        'nb_interactions':["list"],
        'main_category': ["list"],
        'category_hash': ["list"],
        "ts": ["list", "first", "last"],
        "is_test": ["last"],
        "is_search": ["last"],
        "timestamp": ["first"],
        'timestamp_day': ["list"],
        'timestamp_hour': ["list"],
        'timestamp_month': ["list"],
        'timestamp_wd': ["list"],
        'timestamp_age_days': ["list"],
        'timestamp_age_days_norm': ["list"],
        'timestamp_hour_sin': ["list"],
        'timestamp_hour_sin_norm': ["list"],
        'timestamp_hour_cos': ["list"],
        'timestamp_hour_cos_norm': ["list"],
        'timestamp_wd_sin': ["list"],
        'timestamp_wd_sin_norm': ["list"],
        'timestamp_wd_cos': ["list"],
        'timestamp_wd_cos_norm': ["list"],   
        },
    name_sep="-")

### Three workflows 

- workflow 1 : group other features that user interactions 

In [18]:
remaining_columns = [x for x in groupby_other_features.columns if x!= 'timestamp-first']
day_index = ((groupby_other_features - remaining_columns)  >> 
    nvt.ops.LambdaOp(lambda col: (col.max() - col).dt.days + 1) >> 
    nvt.ops.Rename(f = lambda col: "day_index")
)              

In [19]:
workflow = nvt.Workflow(groupby_other_features + day_index)
dataset = nvt.Dataset(interactions_merged_df, cpu=False)
workflow.fit(dataset)
new_gdf_other = workflow.transform(dataset).to_ddf().compute()
len(new_gdf_other)

5077052

In [20]:
new_gdf_other.head(2)

,main_category-list,has_been_removed_from_cart-list,nb_interactions-list,product_action_filled-list,timestamp-first,has_been_clicked-list,category_hash-list,timestamp_wd-list,is_test-last,has_been_added_to_cart-list,...,price_bucket-list,ts-first,mean_price_main-list,is_search-last,timestamp_age_days_norm-list,timestamp_hour_sin-list,timestamp_wd_cos-list,has_been_purchased-list,event_type-list,day_index
0,"[2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1.1184635, -0.17000672, -0.17000672, -0.17000...","[3, 6, 6, 6, 1, 6, 6, 6, 3, 6, 6, 6, 3, 6, 6]",2019-03-12 20:43:11.039,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[87, 0, 0, 0, 87, 0, 0, 0, 87, 0, 0, 0, 87, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0,"[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",...,"[10, 0, 0, 0, 10, 0, 0, 0, 10, 0, 0, 0, 10, 0, 0]",1552423,"[0.042574957, 0.042574957, 0.042574957, 0.0425...",0,"[0.80437595, 0.80437595, 0.80437595, 0.8043759...","[-0.866025, -0.866025, -0.70710653, -0.7071065...","[-0.22252177, -0.22252177, -0.22252177, -0.222...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2]",63
1,"[0, 0, 0, 0, 3]","[0, 0, 0, 0, 0]","[-0.17000672, -0.17000672, -0.17000672, -0.170...","[6, 6, 6, 6, 2]",2019-03-06 22:30:35.813,"[0, 0, 0, 0, 1]","[0, 0, 0, 0, 138]","[2, 2, 2, 2, 2]",0,"[0, 0, 0, 0, 0]",...,"[0, 0, 0, 0, 8]",1551911,"[0.042574957, 0.042574957, 0.042574957, 0.0425...",1,"[0.7084063, 0.7084063, 0.7084063, 0.7084063, 0...","[-0.49999934, -0.49999934, -0.49999934, -0.499...","[-0.90096927, -0.90096927, -0.90096927, -0.900...","[0, 0, 0, 0, 0]","[2, 2, 2, 2, 3]",69


- workflow 2 : create the sequence of product interactions and pageviews

In [21]:
workflow = nvt.Workflow(groupby_product_url)
dataset = nvt.Dataset(interactions_merged_df, cpu=False)
workflow.fit(dataset)
new_gdf_sku_url = workflow.transform(dataset).to_ddf().compute()
len(new_gdf_sku_url)

5077052

In [22]:
new_gdf_sku_url.head()

,session_id_hash,product_url_hash_list,product_url_hash_count
0,1,"[402931, 20527, 32540, 131212, 402931, 453295,...",15
1,2,"[238021, 390783, 131212, 128194, 392871]",5
2,3,"[217635, 114863]",2
3,4,[108255],1
4,5,"[459422, 73300, 97217, 467562, 207892, 441663,...",36


- workflow 3 : create sequence with only product interactions

In [23]:
workflow = nvt.Workflow(groupby_only_product)
dataset = nvt.Dataset(interactions_merged_df, cpu=False)
workflow.fit(dataset)
new_gdf_prod_only = workflow.transform(dataset).to_ddf().compute()
len(new_gdf_prod_only)

5077052

In [24]:
new_gdf_prod_only.head()

,session_id_hash,product_sku_hash_count,product_sku_hash_list
0,1,15,"[47810, 58914, 58914, 58914, 47810, 58914, 589..."
1,2,5,"[58914, 58914, 58914, 58914, 46629]"
2,3,2,"[58914, 58914]"
3,4,1,[58914]
4,5,36,"[54427, 58914, 58914, 55344, 58914, 52381, 589..."


### Merge the three resulting frames 

In [25]:
sessions_gdf = new_gdf_sku_url.merge(new_gdf_other, on='session_id_hash',  how='inner')

In [26]:
sessions_gdf = sessions_gdf.merge(new_gdf_prod_only,  on='session_id_hash',  how='left' )

In [27]:
sessions_gdf.columns

Index(['session_id_hash', 'product_url_hash_list', 'product_url_hash_count',
       'main_category-list', 'has_been_removed_from_cart-list',
       'nb_interactions-list', 'product_action_filled-list', 'timestamp-first',
       'has_been_clicked-list', 'category_hash-list', 'timestamp_wd-list',
       'is_test-last', 'has_been_added_to_cart-list',
       'timestamp_age_days-list', 'has_been_detailed-list',
       'timestamp_hour-list', 'ts-list', 'ts-last', 'timestamp_hour_cos-list',
       'timestamp_wd_sin-list', 'timestamp_day-list',
       'mean_price_hierarchy-list', 'price_bucket-list', 'ts-first',
       'mean_price_main-list', 'is_search-last',
       'timestamp_age_days_norm-list', 'timestamp_hour_sin-list',
       'timestamp_wd_cos-list', 'has_been_purchased-list', 'event_type-list',
       'day_index', 'product_sku_hash_count', 'product_sku_hash_list'],
      dtype='object')

In [28]:
SELECTED_COLS =[ 'session_id_hash', 'day_index', 'product_sku_hash_count', 'product_sku_hash_list',
                 'product_url_hash_count', 'product_url_hash_list',
                 'has_been_removed_from_cart-list', 'has_been_added_to_cart-list',
                 'has_been_purchased-list', 'has_been_detailed-list', 'has_been_clicked-list',
                 'event_type-list', 'product_action_filled-list',  'nb_interactions-list',
                 'category_hash-list', 'main_category-list',
                 'price_bucket-list', 'mean_price_hierarchy-list', 'mean_price_main-list',
                 'ts-first', 'ts-last',  'ts-list', 'day_index',
                 'timestamp_hour_cos-list', 'timestamp_hour_sin-list', 'timestamp_wd_sin-list', 'timestamp_wd_cos-list',
                 'timestamp_age_days-list', 'timestamp_age_days_norm-list', 'is_test-last']
sessions_gdf = sessions_gdf[SELECTED_COLS]
sessions_gdf.head()

,session_id_hash,day_index,product_sku_hash_count,product_sku_hash_list,product_url_hash_count,product_url_hash_list,has_been_removed_from_cart-list,has_been_added_to_cart-list,has_been_purchased-list,has_been_detailed-list,...,ts-first,ts-last,ts-list,timestamp_hour_cos-list,timestamp_hour_sin-list,timestamp_wd_sin-list,timestamp_wd_cos-list,timestamp_age_days-list,timestamp_age_days_norm-list,is_test-last
0,54401,30,21,"[58551, 58914, 17031, 21637, 52446, 58914, 107...",21,"[494446, 212645, 142921, 182250, 442161, 36388...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, ...",...,1555263,1555265,"[1555263, 1555263, 1555263, 1555263, 1555263, ...","[-0.25881854, -0.25881854, -0.25881854, -0.258...","[-0.965926, -0.965926, -0.965926, -0.965926, -...","[1.1285199e-06, 1.1285199e-06, 1.1285199e-06, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[80, 89, 89, 89, 89, 86, 52, 89, 89, 73, 46, 8...","[1.1075752, 1.1984842, 1.1984842, 1.1984842, 1...",0
1,54402,117,2,"[58914, 58914]",2,"[131212, 135345]","[0, 0]","[0, 0]","[0, 0]","[0, 0]",...,1547750,1547750,"[1547750, 1547750]","[4.8876205e-07, 4.8876205e-07]","[-1.0, -1.0]","[-0.43388462, -0.43388462]","[-0.90096843, -0.90096843]","[2, 2]","[-1.7361957, -1.7361957]",0
2,54403,55,1,[58914],1,[140656],[0],[0],[0],[0],...,1553129,1553129,[1553129],[1.0],[2.6179939e-07],[-0.43388462],[-0.90096843],[64],[0.91769767],0
3,54404,107,3,"[58914, 58914, 58914]",3,"[290789, 225684, 54617]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",...,1548637,1548637,"[1548637, 1548637, 1548637]","[0.96592575, 0.96592575, 0.96592575]","[0.25881928, 0.25881928, 0.25881928]","[0.7818321, 0.7818321, 0.7818321]","[0.6234891, 0.6234891, 0.6234891]","[12, 12, 12]","[-0.4709855, -0.4709855, -0.4709855]",0
4,54405,63,11,"[58914, 58914, 58914, 58914, 58914, 58914, 467...",11,"[57922, 396346, 131212, 20145, 111114, 73399, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]",...,1552441,1552442,"[1552441, 1552441, 1552441, 1552441, 1552441, ...","[0.96592575, 0.96592575, 0.96592575, 0.9659257...","[0.25881928, 0.25881928, 0.25881928, 0.2588192...","[0.43388295, 0.43388295, 0.43388295, 0.4338829...","[-0.90096927, -0.90096927, -0.90096927, -0.900...","[56, 56, 56, 56, 56, 56, 48, 48, 48, 56, 55]","[0.80437595, 0.80437595, 0.80437595, 0.8043759...",0


- check day data 

In [29]:
sessions_gdf['day_index'].min(), sessions_gdf['day_index'].max()

(1, 119)

In [30]:
sessions_gdf.groupby('day_index').size().sort_index()

day_index
1        6735
2        5773
3        5133
4        6351
5        5463
        ...  
115    104127
116     93072
117    103282
118     84381
119     56281
Length: 119, dtype: int32

### Remove sessions with one interaction in product_sku_url from train data 

In [31]:
filtered_sessions = sessions_gdf.columns >> \
                     nvt.ops.Filter(f=lambda df: ((df["product_url_hash_count"] >= MINIMUM_SESSION_LENGTH) & (df["is_test-last"] == 0)) | (df["is_test-last"] == 1)) 
workflow = nvt.Workflow(filtered_sessions)
dataset = nvt.Dataset(sessions_gdf, cpu=False)
workflow.fit(dataset)
sessions_gdf = workflow.transform(dataset).to_ddf().compute()
len(sessions_gdf)

3433782

### Create cross-validation folds 

- Define random 5 folds column 

In [32]:
from sklearn.model_selection import GroupKFold
sessions_gdf['fold'] = np.random.randint(1,6, sessions_gdf.shape[0]) 

In [33]:
sessions_gdf.fold.value_counts()

3    688917
4    687053
1    686727
5    686135
2    684950
Name: fold, dtype: int32

- Reserve the 3 last weeks for validation 

In [34]:
sessions_gdf['is_valid'] = 0 
sessions_gdf.loc[((sessions_gdf['is_test-last']==0) & (sessions_gdf['day_index']<=50)), 'is_valid'] = 1

In [35]:
sessions_gdf.is_valid.value_counts()

0    2867164
1     566618
Name: is_valid, dtype: int32

In [36]:
sessions_gdf['is_test-last'].value_counts()

0    3291455
1     142327
Name: is_test-last, dtype: int32

In [37]:
sessions_gdf.head(3)

,session_id_hash,day_index,product_sku_hash_count,product_sku_hash_list,product_url_hash_count,product_url_hash_list,has_been_removed_from_cart-list,has_been_added_to_cart-list,has_been_purchased-list,has_been_detailed-list,...,ts-list,timestamp_hour_cos-list,timestamp_hour_sin-list,timestamp_wd_sin-list,timestamp_wd_cos-list,timestamp_age_days-list,timestamp_age_days_norm-list,is_test-last,fold,is_valid
0,54401,30,21,"[58551, 58914, 17031, 21637, 52446, 58914, 107...",21,"[494446, 212645, 142921, 182250, 442161, 36388...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, ...",...,"[1555263, 1555263, 1555263, 1555263, 1555263, ...","[-0.25881854, -0.25881854, -0.25881854, -0.258...","[-0.965926, -0.965926, -0.965926, -0.965926, -...","[1.1285199e-06, 1.1285199e-06, 1.1285199e-06, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[80, 89, 89, 89, 89, 86, 52, 89, 89, 73, 46, 8...","[1.1075752, 1.1984842, 1.1984842, 1.1984842, 1...",0,2,1
1,54402,117,2,"[58914, 58914]",2,"[131212, 135345]","[0, 0]","[0, 0]","[0, 0]","[0, 0]",...,"[1547750, 1547750]","[4.8876205e-07, 4.8876205e-07]","[-1.0, -1.0]","[-0.43388462, -0.43388462]","[-0.90096843, -0.90096843]","[2, 2]","[-1.7361957, -1.7361957]",0,2,0
2,54404,107,3,"[58914, 58914, 58914]",3,"[290789, 225684, 54617]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",...,"[1548637, 1548637, 1548637]","[0.96592575, 0.96592575, 0.96592575]","[0.25881928, 0.25881928, 0.25881928]","[0.7818321, 0.7818321, 0.7818321]","[0.6234891, 0.6234891, 0.6234891]","[12, 12, 12]","[-0.4709855, -0.4709855, -0.4709855]",0,4,0


### Un-hash session id 

In [38]:
session_map = cudf.read_parquet('./categories/unique.session_id_hash.parquet').reset_index()
session_map

,index,session_id_hash
0,0,<NA>
1,1,00000114e1075962f022114fcfc17f2d874e694ac5d201...
2,2,00000277639fc5c6f816654b78bf3654ece7fd53a7338f...
3,3,000009f36a40de1d557afc083dbb3fc03eef2473337bad...
4,4,00000e812c3076d18245710a31b348d3f23314b7d0dc90...
...,...,...
5077048,5077048,ffffed918e1086333206d96f1fae684de5774866848bb3...
5077049,5077049,fffff2183ad2daa1db3bc4cd320e7248b2f5a6ef9709c4...
5077050,5077050,fffff429d16c1f43796e2c7ccad8a34f8a37b59927b5b4...
5077051,5077051,fffff68e9f346ad171103ae6125899a72c0d77d2d0637c...


## Merge browsing session and search sessions 

In [39]:
session_map.columns = ['session_id_hash', 'original_session_id_hash']
sessions_gdf = sessions_gdf.merge(session_map, on=['session_id_hash'], how='left')

In [40]:
sessions_gdf.head(3)

,session_id_hash,day_index,product_sku_hash_count,product_sku_hash_list,product_url_hash_count,product_url_hash_list,has_been_removed_from_cart-list,has_been_added_to_cart-list,has_been_purchased-list,has_been_detailed-list,...,timestamp_hour_cos-list,timestamp_hour_sin-list,timestamp_wd_sin-list,timestamp_wd_cos-list,timestamp_age_days-list,timestamp_age_days_norm-list,is_test-last,fold,is_valid,original_session_id_hash
0,9188,92,9,"[9042, 58914, 16894, 30056, 58914, 58914, 5891...",9,"[75296, 391147, 141736, 252931, 456758, 394291...","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 1, 1, 0, 0, 0, 0, 0]",...,"[-0.8660252, -0.8660252, -0.8660252, -0.866025...","[-0.50000036, -0.50000036, -0.50000036, -0.500...","[0.7818321, 0.7818321, 0.7818321, 0.7818321, 0...","[0.6234891, 0.6234891, 0.6234891, 0.6234891, 0...","[27, 27, 27, 27, 26, 26, 27, 27, 27]","[0.19103095, 0.19103095, 0.19103095, 0.1910309...",0,4,0,00777f34d40ff8cf318666f38906db9cbf994d50c18fd9...
1,9189,71,4,"[58914, 58914, 58070, 50713]",4,"[429119, 382428, 490517, 427963]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 1, 1]",...,"[-0.96592593, -0.96592593, -0.96592593, -0.965...","[0.2588187, 0.2588187, 0.2588187, 0.2588187]","[0.7818321, 0.7818321, 0.7818321, 0.7818321]","[0.6234891, 0.6234891, 0.6234891, 0.6234891]","[47, 47, 47, 46]","[0.6560972, 0.6560972, 0.6560972, 0.6379316]",0,1,0,00777fe7e4a0ccc24c44ad0377126b216a8e3f54913478...
2,9191,113,3,"[1488, 22468, 58914]",3,"[12281, 189556, 463295]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[1, 1, 0]",...,"[4.8876205e-07, 4.8876205e-07, 4.8876205e-07]","[-1.0, -1.0, -1.0]","[0.7818321, 0.7818321, 0.7818321]","[0.6234891, 0.6234891, 0.6234891]","[6, 6, 6]","[-1.0051154, -1.0051154, -1.0051154]",0,3,0,00778565e985ea074983c8b28d276d92a4a9572af9e549...


### Truncate train and test sessions to last 30 elements

In [41]:
[x for x in sessions_gdf.columns if '_list' in x or '-list' in x ]

['product_sku_hash_list',
 'product_url_hash_list',
 'has_been_removed_from_cart-list',
 'has_been_added_to_cart-list',
 'has_been_purchased-list',
 'has_been_detailed-list',
 'has_been_clicked-list',
 'event_type-list',
 'product_action_filled-list',
 'nb_interactions-list',
 'category_hash-list',
 'main_category-list',
 'price_bucket-list',
 'mean_price_hierarchy-list',
 'mean_price_main-list',
 'ts-list',
 'timestamp_hour_cos-list',
 'timestamp_hour_sin-list',
 'timestamp_wd_sin-list',
 'timestamp_wd_cos-list',
 'timestamp_age_days-list',
 'timestamp_age_days_norm-list']

In [42]:
features_list = [x for x in sessions_gdf.columns if '_list' in x or '-list' in x ]
features_trim =  features_list >> nvt.ops.ListSlice(-30)
other_cols = list(set(sessions_gdf.columns).difference(set(features_list)))
workflow = nvt.Workflow(other_cols + features_trim)
dataset = nvt.Dataset(sessions_gdf[sessions_gdf.is_valid == 0], cpu=False)
workflow.fit(dataset)
trimed_cols = workflow.transform(dataset).to_ddf().compute()
len(trimed_cols)

2867164

In [43]:
trimed_cols.head(2)

,product_sku_hash_list,product_url_hash_list,has_been_removed_from_cart-list,has_been_added_to_cart-list,has_been_purchased-list,has_been_detailed-list,has_been_clicked-list,event_type-list,product_action_filled-list,nb_interactions-list,...,day_index,fold,ts-first,is_valid,product_sku_hash_count,is_test-last,original_session_id_hash,session_id_hash,ts-last,product_url_hash_count
0,"[9042, 58914, 16894, 30056, 58914, 58914, 5891...","[75296, 391147, 141736, 252931, 456758, 394291...","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 1, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 2, 1, 1, 2, 2, 2, 2, 2]","[3, 6, 3, 3, 6, 6, 6, 6, 6]","[-0.17000672, -0.17000672, -0.17000672, -0.170...",...,92,4,1549893,0,9,0,00777f34d40ff8cf318666f38906db9cbf994d50c18fd9...,9188,1549893,9
1,"[58914, 58914, 58070, 50713]","[429119, 382428, 490517, 427963]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 1, 1]","[0, 0, 0, 0]","[2, 2, 1, 1]","[6, 6, 3, 3]","[-0.17000672, -0.17000672, -0.17000672, -0.170...",...,71,1,1551699,0,4,0,00777fe7e4a0ccc24c44ad0377126b216a8e3f54913478...,9189,1551699,4


### Merge back with validation session

In [44]:
final_session_gdf = cudf.concat([sessions_gdf[sessions_gdf.is_valid !=0], trimed_cols])

In [45]:
final_session_gdf.head(3)

,session_id_hash,day_index,product_sku_hash_count,product_sku_hash_list,product_url_hash_count,product_url_hash_list,has_been_removed_from_cart-list,has_been_added_to_cart-list,has_been_purchased-list,has_been_detailed-list,...,timestamp_hour_cos-list,timestamp_hour_sin-list,timestamp_wd_sin-list,timestamp_wd_cos-list,timestamp_age_days-list,timestamp_age_days_norm-list,is_test-last,fold,is_valid,original_session_id_hash
6,9198,31,3,"[58914, 58914, 58914]",3,"[250602, 176004, 32540]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",...,"[-0.4999995, -0.4999995, -0.4999995]","[-0.8660257, -0.8660257, -0.8660257]","[-0.781831, -0.781831, -0.781831]","[0.6234904, 0.6234904, 0.6234904]","[88, 88, 88]","[1.1888435, 1.1888435, 1.1888435]",0,5,1,00778adaef6807c6598f0991039ed30780e97a67b13728...
8,9205,35,2,"[58914, 39336]",2,"[32430, 330683]","[0, 0]","[0, 0]","[0, 0]","[0, 1]",...,"[-0.96592575, -0.96592575]","[-0.2588193, -0.2588193]","[0.9749277, 0.9749277]","[-0.22252177, -0.22252177]","[83, 83]","[1.1389544, 1.1389544]",0,1,1,00779b2c15725c8a398b8a594670fb68c0668fce31930c...
16,9216,37,2,"[22912, 58914]",2,"[193086, 334664]","[0, 0]","[0, 0]","[0, 0]","[1, 0]",...,"[-0.25881854, -0.25881854]","[-0.965926, -0.965926]","[1.1285199e-06, 1.1285199e-06]","[1.0, 1.0]","[44, 44]","[0.60041094, 0.60041094]",0,1,1,0077b5c8d26fd156622c9e5d51b98fc9b28e9f683ac32f...


In [48]:
final_session_gdf.shape

(3433782, 32)

### Add search context 

In [49]:
search_table = cudf.read_parquet(OUTPUT_DIR+"/session_search.parquet")


In [50]:
search_table.columns  = ['original_session_id_hash', 'flat_query_vector', 'flat_product_skus_hash',
       'flat_clicked_skus_hash', 'impressions_size', 'clicks_size','nb_queries',
       'clicked-flag']

In [51]:
merged_session_table = final_session_gdf.merge(search_table, on=['original_session_id_hash'], how='left')

In [52]:
del final_session_gdf, search_table

# Process missing values 

In [53]:
merged_session_table = merged_session_table.to_pandas()

In [54]:
merged_session_table.loc[merged_session_table.flat_product_skus_hash.isna(), 
                         'flat_product_skus_hash'] = pd.Series([['missing']] * merged_session_table.flat_product_skus_hash.isna().sum()).values

In [55]:
merged_session_table.loc[merged_session_table.flat_clicked_skus_hash.isnull(),
                         'flat_clicked_skus_hash'] = pd.Series([['missing']] * merged_session_table.flat_clicked_skus_hash.isnull().sum()).values

In [56]:
merged_session_table.loc[merged_session_table.flat_query_vector.isnull(),
                         'flat_query_vector'] = pd.Series([np.zeros(50)] * merged_session_table.flat_query_vector.isnull().sum()).values

In [57]:
merged_session_table.loc[merged_session_table.impressions_size.isnull(),
                         'impressions_size'] = pd.Series([[0]] * merged_session_table.impressions_size.isnull().sum()).values

In [58]:
merged_session_table.nb_queries.fillna(0, inplace=True)

In [59]:
merged_session_table.head(5)

,session_id_hash,day_index,product_sku_hash_count,product_sku_hash_list,product_url_hash_count,product_url_hash_list,has_been_removed_from_cart-list,has_been_added_to_cart-list,has_been_purchased-list,has_been_detailed-list,...,fold,is_valid,original_session_id_hash,flat_query_vector,flat_product_skus_hash,flat_clicked_skus_hash,impressions_size,clicks_size,nb_queries,clicked-flag
0,156516,42,2,"[58914, 4918]",2,"[10523, 41301]","[0, 0]","[0, 0]","[0, 0]","[0, 1]",...,1,1,07de4f39c1069c272a8d12d0ed2a9b47664f0c96c97090...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[missing],[missing],[0],None,0.0,None
1,87496,33,13,"[26337, 58914, 58914, 12540, 58914, 45977, 431...",13,"[222523, 32540, 117702, 105356, 262542, 387012...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1]",...,2,1,046a84c172dd202eb3d9216328d9366e517d02a6487f31...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[missing],[missing],[0],None,0.0,None
2,87499,41,6,"[58914, 58914, 58914, 58914, 58914, 58914]",6,"[42826, 392119, 303925, 216092, 445682, 410243]","[0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0]",...,5,1,046a925a4a82d22b0dc1c8bdc76d3bee31c376429315c9...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[missing],[missing],[0],None,0.0,None
3,139738,30,6,"[41698, 49083, 49083, 58914, 58914, 58914]",6,"[350821, 413931, 413931, 234964, 20934, 111114]","[0, 0, 0, 0, 0, 0]","[0, 1, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0]","[1, 1, 1, 0, 0, 0]",...,4,1,0707ed803d391d72ab6a773cd29977344979da650618a3...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[missing],[missing],[0],None,0.0,None
4,139741,46,4,"[58914, 58914, 58914, 29534]",4,"[398545, 58420, 479220, 248619]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 1]",...,1,1,0707f4ae9aae359e8998cd55a5efe44581327dc641e0b2...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[missing],[missing],[0],None,0.0,None


### Encode flat_product_skus_hash and flat_clicked_skus_hash using product_url_hash mapping

- load mapping 

In [60]:
mapping = pd.read_parquet('./categories/unique.product_url_hash.parquet')

In [61]:
mapping_dict = dict(zip(mapping.product_url_hash,mapping.index))

- Update mapping with unseen browsing products ids present in `flat_product_skus_hash` and `flat_clicked_skus_hash`

In [62]:
prods = [e for impression in merged_session_table.flat_product_skus_hash.values for e in impression]
clicked_prod = [e for impression in merged_session_table.flat_clicked_skus_hash.values for e in impression]
all_prods = set(prods + clicked_prod)
new_prods = all_prods.difference(set(mapping.product_url_hash))
print('Number of products present in search and not in browsing is: %s' %len(new_prods))

Number of products present in search and not in browsing is: 968


In [63]:
cardinality = len(mapping_dict)
new_ids = list(range(cardinality, cardinality+len(new_prods)))
new_dict = dict(zip(new_prods, new_ids))
mapping_dict.update(new_dict)

- Encode list columns :  `flat_product_skus_hash` and `flat_clicked_skus_hash`

In [64]:
def encode_list(x): 
    return [mapping_dict[e] for e in x]

In [65]:
merged_session_table.flat_product_skus_hash = merged_session_table.flat_product_skus_hash.progress_apply(encode_list)
merged_session_table.flat_clicked_skus_hash = merged_session_table.flat_clicked_skus_hash.progress_apply(encode_list)

100%|██████████| 3433782/3433782 [00:09<00:00, 374857.26it/s]


- Save updated mapping 

In [104]:
pd.DataFrame(mapping_dict.keys(), columns=['product_url_hash']).to_parquet(os.path.join(OUTPUT_DIR,
                                                                                        'unique.updated_product_url_hash.parquet'))

### Create clicked-flag : 

- check if element of `flat_product_skus_hash` was clicked or not.

P.S: We ignore the click if interacted item is in the list of clicks (True in 1% of search interactions). 

In [67]:
merged_session_table['clicked-flag'] = merged_session_table.progress_apply(lambda x: [int(e in  list(set(x['flat_clicked_skus_hash']).difference(set(x['product_sku_hash_list'])))) \
                                                                          for e in x['flat_product_skus_hash']], axis=1)

100%|██████████| 3433782/3433782 [03:38<00:00, 15686.10it/s]


In [68]:
merged_session_table[1020:1030]

,session_id_hash,day_index,product_sku_hash_count,product_sku_hash_list,product_url_hash_count,product_url_hash_list,has_been_removed_from_cart-list,has_been_added_to_cart-list,has_been_purchased-list,has_been_detailed-list,...,fold,is_valid,original_session_id_hash,flat_query_vector,flat_product_skus_hash,flat_clicked_skus_hash,impressions_size,clicks_size,nb_queries,clicked-flag
1020,105340,50,2,"[58914, 58914]",2,"[301681, 131212]","[0, 0]","[0, 0]","[0, 0]","[0, 0]",...,4,1,054dfd808a4f4bf812717b3a5d4d8bae0f309ff0ec290d...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[497504],[497504],[0],None,0.0,[1]
1021,17221,36,2,"[40639, 58914]",2,"[341770, 383815]","[0, 0]","[0, 0]","[0, 0]","[1, 0]",...,2,1,00e03b33868ca347f748b8e110f8aa4396f150b304280d...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[497504],[497504],[0],None,0.0,[1]
1022,17225,37,2,"[58914, 10318]",2,"[441472, 86447]","[0, 0]","[0, 0]","[0, 0]","[0, 1]",...,3,1,00e04835a7e12dc922ad329d18ba75b961f7942d205e2c...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[497504],[497504],[0],None,0.0,[1]
1023,17245,49,2,"[16999, 42918]",2,"[142667, 361053]","[0, 0]","[0, 0]","[0, 0]","[1, 1]",...,4,1,00e08811a99b2eb1bb6cb8248eb37058667f60066078c7...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[497504],[497504],[0],None,0.0,[1]
1024,161491,38,16,"[44245, 32040, 58914, 35486, 52683, 58914, 589...",16,"[372268, 269971, 369425, 298312, 444336, 26225...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0]",...,3,1,081de81fa9babf9b8604a597dd09eb0f47d95204b59280...,"[-0.17531707882881165, -0.010863647796213627, ...","[372268, 433967, 331697, 379410, 439998, 26389...",[372268],[25.0],[1.0],1.0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1025,161502,50,4,"[58914, 58914, 58914, 58914]",4,"[233981, 131212, 256998, 324338]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]",...,2,1,081e05c3ea4b87127077814cf0ad8778a0c732b21c2c59...,"[-0.018799802288413048, -0.2817137837409973, 0...",[497504],[497504],"[0.0, 0.0]","[0.0, 0.0]",2.0,[1]
1026,249221,36,2,"[58914, 58914]",2,"[477245, 131212]","[0, 0]","[0, 0]","[0, 0]","[0, 0]",...,2,1,0c893615eb0ebc720afb67723833cc002044a40d8895f6...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[497504],[497504],[0],None,0.0,[1]
1027,249224,32,71,"[591, 25918, 2077, 58914, 58914, 58914, 58914,...",71,"[4858, 218796, 17294, 301681, 205428, 308995, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, ...",...,5,1,0c89483cff73ccd3670562508fd3633433c90d9af8d993...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[497504],[497504],[0],None,0.0,[1]
1028,249231,49,3,"[58914, 58914, 24599]",3,"[185635, 131212, 207718]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 1]",...,1,1,0c895d93d348dbbbdaa0ad2297f1c68889643d318af9a9...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[497504],[497504],[0],None,0.0,[1]
1029,227542,39,2,"[58914, 58914]",2,"[32643, 319985]","[0, 0]","[0, 0]","[0, 0]","[0, 0]",...,2,1,0b6fdc64879b7139e9d60bd2cf64eebed726585ac108a9...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[497504],[497504],[0],None,0.0,[1]


In [107]:
merged_session_table['is_test-last'].sum()

142327

### Exporting session table by fold and train-test-valid splits 

In [70]:
OUTPUT_FOLDER = "/result/coveo/"
for fold in range(1, 6): 
    merged_session_table.loc[(merged_session_table['is_test-last']==1) & (merged_session_table.fold==fold)].to_parquet(os.path.join(OUTPUT_DIR, 'test-%s.parquet'%fold),
                                                                                               #partition_file_name= 'test-%s.parquet'%fold,
                                                                                               #engine='pyarrow',
                                                                                               #row_group_size=1000
                                                                                                           )
    
    
    
    merged_session_table.loc[(merged_session_table['is_valid']==1) & (merged_session_table.fold==fold)].to_parquet(os.path.join(OUTPUT_DIR, 'valid-%s.parquet'%fold),
                                                                                           #partition_file_name = 'valid-%s.parquet'%fold,
                                                                                           #engine='pyarrow',
                                                                                           #row_group_size=1000
                                                                                                       )
    
    
    merged_session_table.loc[(merged_session_table['is_valid']==0) & (merged_session_table['is_test-last']==0) & (merged_session_table.fold==fold)].to_parquet(os.path.join(OUTPUT_DIR, 'train-%s.parquet'%fold),
                                                                                           #partition_file_name = 'train-%s.parquet'%fold,
                                                                                           #engine='pyarrow',
                                                                                           #row_group_size=1000
                                                                                                                                           )

In [71]:
test = pd.read_parquet(os.path.join(OUTPUT_DIR, "test-1.parquet"))

In [72]:
test.head(2)

,session_id_hash,day_index,product_sku_hash_count,product_sku_hash_list,product_url_hash_count,product_url_hash_list,has_been_removed_from_cart-list,has_been_added_to_cart-list,has_been_purchased-list,has_been_detailed-list,...,fold,is_valid,original_session_id_hash,flat_query_vector,flat_product_skus_hash,flat_clicked_skus_hash,impressions_size,clicks_size,nb_queries,clicked-flag
522702,66809,7,3,"[58914, 58914, 58914]",3,"[391147, 131212, 47979]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",...,1,0,035f8723c5377d6adf48e6509fa5762b5bcf4a37810d44...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[497504],[497504],[0.0],None,0.0,[1]
539462,160794,2,1,[56108],1,[473839],[0],[0],[0],[1],...,1,0,0814d1be32e549b7651837e189b8d19e9cc8aeb4a4ee92...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[497504],[497504],[0.0],None,0.0,[1]


### Save unique product sku mapping from updated product_url_hash encoded column

In [73]:
urls_ids = interactions_merged_df[interactions_merged_df.event_type==2]['product_url_hash'].unique()

In [74]:
mapping = pd.read_parquet(os.path.join(OUTPUT_DIR, 'unique.updated_product_url_hash.parquet'))

In [75]:
mask = mapping.reset_index()['index'].isin(urls_ids.values.tolist())

In [76]:
mapping_prod = mapping[~mask].reset_index()

In [77]:
mapping_prod.columns =  ['encoded_product_sku', 'original_product_sku']

In [78]:
mapping_prod.head()

,encoded_product_sku,original_product_sku
0,0,None
1,3,000082c13a39a85caafc0b435f37b8cccb3aa3b0c63d56...
2,15,00021c22af5a39b6297fa304566f9cfcf7982689904a0b...
3,18,000287bdd487f7ca56435b6af745222a3e2c3452ca9d53...
4,36,00056f966469cf20e4d2c8ecd863e54c613161e57960d7...


In [79]:
mapping_prod.shape[0]

59883

In [88]:
mapping_prod.to_parquet(os.path.join(OUTPUT_DIR, 'mapping_product_sku_without_urls.parquet'))

### Create pickle files with product embedding vectors 

* Pickle file with a tuple composed by:

    - Numpy matrix with the description vectors of the products
    - Numpy matrix with the image vectors of the products
    - Dict to map the encoded product_sku to the position in the embedding matrices

- Load product table 

In [82]:
product_info = pd.read_csv('/workspace/sku_to_content.csv', usecols=['product_sku_hash', 
                                                                     'description_vector', 
                                                                     'image_vector'])
product_info.tail() 

,product_sku_hash,description_vector,image_vector
66381,c7cc673ca3baa5fa222fffdc16379892b3a62583a48143...,NaN,NaN
66382,6641c7d2053ce48ce1e81a9653dffe56dbb79ab0704fbd...,"[-0.19150441884994507, -0.06235162168741226, -...","[129.55668732976045, 43.27996741934932, -36.70..."
66383,526a6a51717d5bb40ef2b0c47394d08c54385375633bab...,"[-0.19760936498641968, 0.4446450471878052, -0....","[-158.25984189321855, 74.19255741438077, 199.6..."
66384,21ca4ab0e2fbd3b401fbeadeb4439dcab9998fb52159ec...,NaN,NaN
66385,0eaa39fb645749da181c0637d3b420f3f103b5f6b50286...,NaN,NaN


In [83]:
# convert strings to list object 
import ast
def convert_str_to_list(x): 
    if pd.isnull(x): 
        return x
    return ast.literal_eval(x)
for col in ['description_vector', 'image_vector']: 
    product_info[col] = product_info[col].progress_apply(convert_str_to_list)

100%|██████████| 66386/66386 [00:05<00:00, 11560.31it/s]


In [84]:
product_info.columns = ['original_product_sku', 'description_vector', 'image_vector']

### Merge product embeddings and mapping_prod

In [85]:
embeddings_table = mapping_prod.merge(product_info, on=['original_product_sku'], how='left')

- Fill missing embeddings with vector of zeros 

In [86]:
embeddings_table.loc[embeddings_table.description_vector.isnull(),
                         'description_vector'] = pd.Series([np.zeros(50)] * embeddings_table.description_vector.isnull().sum()).values

embeddings_table.loc[embeddings_table.image_vector.isnull(),
                         'image_vector'] = pd.Series([np.zeros(50)] * embeddings_table.image_vector.isnull().sum()).values

In [87]:
embeddings_table

,encoded_product_sku,original_product_sku,description_vector,image_vector
0,0,None,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,3,000082c13a39a85caafc0b435f37b8cccb3aa3b0c63d56...,"[0.1271490901708603, 0.09000279754400253, -0.1...","[79.61198119384494, 73.19961102682859, -92.346..."
2,15,00021c22af5a39b6297fa304566f9cfcf7982689904a0b...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,18,000287bdd487f7ca56435b6af745222a3e2c3452ca9d53...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,36,00056f966469cf20e4d2c8ecd863e54c613161e57960d7...,"[0.33688265085220337, 0.03285939246416092, -0....","[145.95916885879836, 37.90878066571861, -85.77..."
...,...,...,...,...
59878,498468,9acd34249a1a9e13753d39e2adc61a6c4a82a7d743a634...,"[-0.21348106861114502, -0.18012449145317078, 0...","[83.05163496671773, -80.84915746650896, -39.86..."
59879,498469,c4b2c170182d1abdcdf4dafc233ce0c2124786cdd86f79...,"[-0.1523011028766632, 0.35209035873413086, 0.0...","[-139.49862168470486, 142.81849208243293, 154...."
59880,498470,76142c18e8ae06e682a2044ed464f1d3cb43ccfade37f1...,"[-0.1972847580909729, -0.12598703801631927, -0...","[-96.79500596434526, -24.44535110386083, 53.20..."
59881,498471,16bbcd368b7fb9a7dda6e85662ee6f945ceb31b8e6c8b4...,"[-0.2526131868362427, -0.15291501581668854, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


- Numpy matrix with the image vectors of the products

In [89]:
image_matrix = np.concatenate(embeddings_table.image_vector.values).reshape(-1, 50)

- Numpy matrix with the description vectors of the products

In [90]:
desc_matrix = np.concatenate(embeddings_table.description_vector.values).reshape(-1, 50)

- Dict to map the encoded product_sku to the position in the embedding matrices

In [91]:
mapping_id_sku_emb_position = dict(zip(embeddings_table.encoded_product_sku, embeddings_table.index))

### Save to pickle file 

In [93]:
# Saving the objects:
import pickle
with open(os.path.join(OUTPUT_DIR, 'embedding_data.pkl'), 'wb')as f:  
    pickle.dump([desc_matrix, image_matrix, mapping_id_sku_emb_position], f)